### A logistic regression forward pass 

In [125]:
import torch.nn.functional as F
import torch
from torch.autograd import grad

In [126]:
y= torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0] , requires_grad=True)

In [127]:
# compute in an output layer
z = x1*w1+b
a = torch.sigmoid(z)

In [128]:
a, z

(tensor([0.9183], grad_fn=<SigmoidBackward0>),
 tensor([2.4200], grad_fn=<AddBackward0>))

In [129]:
loss = F.binary_cross_entropy(a,y)

In [130]:
loss

tensor(0.0852, grad_fn=<BinaryCrossEntropyBackward0>)

### Updating w1 and b

In [131]:
grad_L_w1 = grad(loss, w1, retain_graph=True)
grad_L_b = grad(loss, b, retain_graph=True)

In [132]:
print(grad_L_w1)
print(grad_L_b)

(tensor([-0.0898]),)
(tensor([-0.0817]),)


### Multilayer perceptron with two hidden layers

In [133]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs:int, num_output:int):
        super().__init__()
        self.layers = torch.nn.Sequential(
            #1st hidden layers
            torch.nn.Linear(num_inputs,30), 
            torch.nn.ReLU(),

            # 2nd hidden layer
            torch.nn.Linear(30,20),
            torch.nn.ReLU(),

            # Output layer 
            torch.nn.Linear(20, num_output)
        )

    def forward(self, x):
        logits = self.layers(x)
        return logits

In [134]:
# Instantiation of new NeuralNetwork model
model = NeuralNetwork(num_inputs=50, num_output=3)

In [135]:
# Printing our model summary 
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [136]:
# Checking for total number of trainable parameters of our model 
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad==True)
print("Total number of trainable parametres are: ", num_params)

Total number of trainable parametres are:  2213


In [137]:
# we can access the corresponding weight parameter matrix(of a specifique layer) as follows
print(model.layers[0].weight[0])

tensor([-0.1205,  0.1037, -0.1028, -0.1124, -0.0894,  0.0640, -0.0523,  0.0529,
        -0.1200, -0.0858, -0.0519, -0.0278, -0.1079,  0.0926, -0.0334,  0.0454,
         0.1000,  0.0263,  0.0387,  0.1365, -0.0638,  0.0448, -0.0629,  0.1011,
         0.1129, -0.1304,  0.1207,  0.0675,  0.0616,  0.0582,  0.1176, -0.0187,
        -0.1196, -0.0406, -0.0996,  0.0093, -0.0264, -0.0759, -0.0129,  0.1340,
        -0.0112,  0.0045, -0.0221,  0.0222,  0.1260,  0.0865,  0.0502,  0.0307,
         0.0333,  0.0546], grad_fn=<SelectBackward0>)


In [138]:
print(model.layers[0].bias)

Parameter containing:
tensor([ 0.0509,  0.0545, -0.0393,  0.0924, -0.1412, -0.1232, -0.1063,  0.0081,
        -0.1249,  0.0101, -0.0019, -0.1298,  0.1388, -0.0330,  0.1017,  0.1247,
        -0.0554, -0.0417,  0.1388,  0.0159,  0.1215,  0.0385,  0.0769, -0.1224,
        -0.0279,  0.0991, -0.0079, -0.1229,  0.0148,  0.0132],
       requires_grad=True)


In [139]:
# Generate random training data 
X = torch.rand((1,50))
X

tensor([[0.3306, 0.5374, 0.2845, 0.8459, 0.2232, 0.2083, 0.8169, 0.1084, 0.3285,
         0.7185, 0.3624, 0.3084, 0.8893, 0.4179, 0.9741, 0.3697, 0.2397, 0.8936,
         0.1443, 0.1365, 0.7625, 0.1632, 0.6641, 0.1525, 0.9830, 0.5936, 0.9120,
         0.0146, 0.6323, 0.4743, 0.7467, 0.3545, 0.9994, 0.9815, 0.7399, 0.2057,
         0.8742, 0.0138, 0.7676, 0.7481, 0.7570, 0.6432, 0.9111, 0.2246, 0.8668,
         0.6961, 0.5131, 0.6978, 0.4537, 0.9035]])

In [140]:
# training the model (Execute forward pass of a model)
out = model(X)
out

tensor([[-0.1152,  0.1256,  0.1569]], grad_fn=<AddmmBackward0>)

In [141]:
# This tells PyTorch that it doesn't need to keep track of the
# gradients, which can result in significant savings in memory and
# computation.

with torch.no_grad():
    out = model(X)
print(out)

tensor([[-0.1152,  0.1256,  0.1569]])


In [142]:
# Using softmax fonction as the activation function 
with torch.no_grad():
    out =torch.softmax(model(X),dim=1)
print(out)

tensor([[0.2789, 0.3549, 0.3662]])


### Setting up efficient data loaders

In [143]:
# Creating a small toy dataset

# Train data 
X_train = torch.tensor([
    [-1.2,3.1],
    [-0.9,2.9],
    [-0.5,2.6],
    [2.3,-1.1],
    [2.7,-1.5],
])

y_train = torch.tensor([0,0,0,1,1])

# test data
X_test = torch.tensor([
    [-0.8,2.8],
    [2.6,-1.6],
    ])

y_test = torch.tensor([0,1])

This Custom ToyDataset class's purpose is to use it to instantiate a pytorch DataLoader.

In [144]:
# creating the custom Dataset class
from torch.utils.data import Dataset

class ToyDataset(Dataset):
    def __init__(self, X , y):
        self.feature = X
        self.labels = y

    def __getitem__(self, index):
        one_x = self.feature[index]
        one_y = self.labels[index]
        return one_x, one_y
    
    def __len__(self):
        return self.labels.shape[0]

In [145]:
train_ds = ToyDataset(X_train,y_train)
test_ds = ToyDataset(X_test, y_test)

In [148]:
# Instantiating data loaders 
from torch.utils.data import DataLoader

torch.manual_seed(123)

# train DataLoader
train_loader = DataLoader(
    dataset=train_ds,
    batch_size= 2,
    shuffle=True,
    num_workers=0
)

# Test DataLoader 
test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0
)

In [150]:
# iterate over the train_loader
for idx, (x,y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x , y)
    print("\n")

Batch 1: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])


Batch 2: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])


Batch 3: tensor([[ 2.7000, -1.5000]]) tensor([1])




This is desired to prevent deep neural networks getting caught in repetitive update
cycles during training. To prevent this, it's recommended to set drop_last=True, which
will drop the last batch in each epoch, as shown below:

In [151]:
# train DataLoader
train_loader = DataLoader(
    dataset=train_ds,
    batch_size= 2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

In [153]:
# iterate once more over the train_loader
for idx, (x,y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x , y)
    print("\n")

Batch 1: tensor([[ 2.3000, -1.1000],
        [ 2.7000, -1.5000]]) tensor([1, 1])


Batch 2: tensor([[-1.2000,  3.1000],
        [-0.9000,  2.9000]]) tensor([0, 0])


